## 初始安裝seleuinm (每次使用 Colab 都要先跑這幾行指令)

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

## 瀏覽器&套件設定

In [1]:
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

In [2]:
# 擋掉通知黑屏，以防止瀏覽器第一次登入時跳出通知視窗
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.notifications": 2
})
options.add_argument('--headless') # colab上須加上headless，表不使用GUI背景執行
options.add_argument('--no-sandbox')
options.headless = True

## 自動登入

In [3]:
# ------ 登入的帳號與密碼 ------
username = str(input("請輸入帳號："))
password = str(input("請輸入密碼："))

driver = webdriver.Chrome(options=options)

url = 'https://www.facebook.com'
driver.get(url)

WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//*[@id="email"]')))

elem = driver.find_element(By.ID,"email")
elem.send_keys(username)

elem = driver.find_element(By.ID,"pass")
elem.send_keys(password)


# 提交登入
elem.send_keys(Keys.RETURN)
time.sleep(5)

# 檢查有沒有被擋下來
if len(driver.find_elements(By.XPATH,"//*[contains(text(), '你的帳號暫時被鎖住')]")) > 0:
    driver.find_elements(By.XPATH,"//*[contains(text(), '是')]")[1].click()
elif len(driver.find_elements(By.XPATH,"//*[contains(text(), '尋找你的帳號並登入。')]")) > 0:
    print("登入失敗，請檢查帳號密碼是否正確")
else:
    wait = WebDriverWait(driver, 10)
    feed = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="feed"]'))) # 等待動態頁面(鎖定貼文)加載完成
    print("登入成功")

登入成功


## 純貼文爬取

In [1]:
posts_number = int(input("請輸入要爬取的貼文數量:"))
posts = [] 


# 初始設定
last_height = driver.execute_script("return document.body.scrollHeight") # 取得頁面高度初始值
last_count = 0 # 計算目前貼文數量

feed = driver.find_element(By.CSS_SELECTOR, 'div[role="feed"]') # 找到動態頁面(鎖定貼文)
while True:


    # 如果已經找到足夠的貼文，則停止
    if len(posts) >= posts_number:
        posts = posts[:posts_number]
        break
    
    # 貼文文字的 class selector
    post_text_class_selector = ".x1iorvi4.x1pi30zi.x1l90r2v.x1swvt13"

    # 貼文中(查看更多)按鈕的 xpath selector
    # post_more_button_xpath_selector = "//div[@class='x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 xggy1nq x1a2a7pz xt0b8zv x1hl2dhg xzsf02u x1s688f' and @role='button']"

    # 捕捉所有的 data-pagelet 元素
    elements = feed.find_elements(By.CSS_SELECTOR, '[data-pagelet]')
    current_count = len(elements)
    if current_count != last_count:
        # 只處理新出現的 elements
        new_elements = elements[last_count:]
        for element in new_elements:
            print('--------------貼文分隔線--------------')
            try:
                # 得知element的html
                # print(element.get_attribute('outerHTML'))
                # button = element.find_element(By.XPATH,post_more_button_xpath_selector).click()
                button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//*[contains(text(), '查看更多')]")))
                print(button)
                button.click()
                print('查看更多 Button clicked!')
                print('---')
            except Exception as e:
                # print('此篇為普通貼文，直接處理 or 按到推薦朋友的按紐，待處理')
                print('---')
            try:
                element = element.find_element(By.CSS_SELECTOR, post_text_class_selector)
                posts.append(element.text)
                print(element.text)
            except:
                print('此篇非普通貼文，跳過處理')

        # 更新 last_count
        last_count = current_count

    # 計算新的滾動高度並比較是否到達底部
    new_height = driver.execute_script("return document.body.scrollHeight")
    last_height = new_height

    # 滾動到頁面底部
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 等待新內容加載(秒)
    time.sleep(6)
print('共爬取', len(posts), '篇貼文')
print(posts)
driver.refresh()

ValueError: invalid literal for int() with base 10: ''

In [ ]:
driver.quit()